In [1824]:
from typing import List, Tuple
import random
import math
from termcolor import colored

## Parameters

In [1825]:
rows = 24
cols = 24
avg = (rows + cols)/2
number_of_region = int(0.000966*avg**2 + 0.31935*avg + 5/3)
number_of_prison = int(-0.0008345*avg**2 + 0.1467*avg + 1.06)
print('Number of regions:',number_of_region)
print('Number of prisons:', number_of_prison)

init_land_chance = 0.1          #Probability of cells to be seeded as 'land terrain', range 0 to 1
init_sea_chance = 0.04          #Probability of cells to be seeded as 'sea terrain', range 0 to 1
init_mountain_chance = 0.04     #Probability of 'land terrain' cells to be seeded as 'mountain terrain', range 0 to 1
init_region_chance = 0.0007     #Probability of 'land terrain' cells to be seeded as a region seed, range 0 to 1

land_chance = 0.055             #Probability of cells to become 'land terrain' for each adjacent 'land terrain' cell, range 0 to 1
sea_chance = 0.022              #Probability of cells to become 'sea terrain' for each adjacent 'sea terrain' cell, range 0 to 1
border_sea_chance = 0.08        #Probability of cells on the edges of the map to become 'sea terrain', range -1 to 1
mountain_chance = 0.01          #Probability of 'land terrain' cells to become 'mountain terrain' for each adjacent 'mountain terrain' cell, range 0 to 1
sea_mountain_chance = -0.01     #Probability of 'land terrain' cells to become 'mountain terrain' for each adjacent 'sea terrain' cell, range -1 to 1
mountain_amplifier = 12         #Increase to have larger mountain ranges, range >= 0


Number of regions: 9
Number of prisons: 4


## Helper Functions

In [1826]:
def neighbors(a, radius, row_number, column_number):
     return [[a[i][j] if  i >= 0 and i < len(a) and j >= 0 and j < len(a[0]) else '|'
                for j in range(column_number-radius, column_number+radius+1)]
                    for i in range(row_number-radius, row_number+radius+1)]

In [1827]:
def map_print(Map):
    for coord_x, row in enumerate(Map):
        for coord_y, terrain in enumerate(row):
            cur = Map[coord_x][coord_y]
            if cur == '_' or cur in range(1, number_of_region + 1):                
                print('\033[92m', cur, end = '  ')
            elif cur == '~':                
                print('\033[96m', cur, end = '  ')
            elif cur == 'M':                
                print('\033[91m', cur, end = '  ')
            elif cur == 'p':                
                print('\033[93m', cur, end = '  ')
            else:
                print('\033[97m', cur, end = '  ')
        print()

In [1828]:
def get_neighbour_terrain(area):
    land = 0
    sea = 0
    for coord_x, row in enumerate(area):
        for coord_y, terrain in enumerate(row):
            if area[coord_x][coord_y] == '_':
                land += land_chance
            elif area[coord_x][coord_y] == '~':
                sea += sea_chance
            elif(area[coord_x][coord_y] == '|'):
                sea += border_sea_chance
    
    chance = random.uniform(0,1)
    if chance <= land:
        return '_'
    elif chance >= 1 - sea:
        return '~'
    else:
        return '.'

In [1829]:
def get_neighbour_mountain(area):
    mountain = 0
    for coord_x, row in enumerate(area):
        for coord_y, terrain in enumerate(row):
            if area[coord_x][coord_y] == 'M':
                mountain += mountain_chance
            if area[coord_x][coord_y] == '~':
                mountain += sea_mountain_chance
    
    chance = random.uniform(0,1)
    if chance <= mountain:
        return 'M'
    else:
        return '_'

In [1830]:
def get_neighbour_region(area):
    region = [0 for i in range(number_of_region)]
    for coord_x, row in enumerate(area):
        for coord_y, terrain in enumerate(row):
            if area[coord_x][coord_y] in range(1, number_of_region + 1):
                num = area[coord_x][coord_y]
                region[num - 1] += 0.1
            
    
    chance = random.uniform(0,1)
    for r, c in enumerate(region):
        chance -= c
        if(chance <= 0):
            return r + 1

    return '_'
    

In [1831]:
a = [0, 0, 0.1, 0.3, 0.1, 0, 0.2]
print(sorted(a, key=float))

[0, 0, 0, 0.1, 0.1, 0.2, 0.3]


## Terrain Seeding

In [1832]:
Map = [['.']*rows for _ in range(cols)]

for coord_x, row in enumerate(Map):
    for coord_y, terrain in enumerate(row):
        chance = random.uniform(0,1)
        if chance <= init_land_chance:
            Map[coord_x][coord_y] = '_'
        elif random.uniform(0,1) >= 1 - init_sea_chance:
            Map[coord_x][coord_y] = '~'

In [1833]:
map_print(Map)

 .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   ~   _   .   .   _   _   .   .  
 .   .   .   .   .   _   .   _   .   .   .   .   _   _   .   .   _   .   .   .   .   .   .   .  
 .   _   .   .   .   .   .   _   .   .   .   .   .   .   .   _   .   .   .   .   .   .   .   .  
 _   .   .   .   .   .   .   .   ~   .   .   .   .   .   .   .   .   .   _   .   .   .   .   .  
 .   ~   .   _   .   .   .   _   .   .   .   .   .   .   _   .   .   .   _   ~   .   .   .   .  
 .   .   .   _   .   .   .   .   .   .   _   .   .   .   .   .   .   .   .   .   ~   ~   _   .  
 .   .   .   .   .   _   .   _   .   _   .   .   .   .   _   ~   .   .   _   .   .   _   _   .  
 .   _   .   .   _   .   .   .   .   .   .   .   .   _   .   .   .   _   .   .   .   .   _   .  
 .   .   .   ~   .   .   .   .   .   .   .   .   .   .   ~   .   .   .   .   ~   .   .   ~   .  
 .   .   _   .   .   .   .   .   .   .   .   .   .   _   .   .   .   .   .   .   _   .   .   .  
 _   .   .   .   .   .   .   _

## Land and Sea Filling

In [1834]:
isFull = False

while(isFull == False):
    isFull = True
    for coord_x, row in enumerate(Map):
        for coord_y, terrain in enumerate(row):
            if Map[coord_x][coord_y] == '.':          
                isFull = False
                area = neighbors(Map, 1, coord_x, coord_y).copy()
                Map[coord_x][coord_y] = get_neighbour_terrain(area)            

In [1835]:
map_print(Map)

 ~   ~   ~   ~   _   ~   ~   ~   ~   ~   ~   ~   ~   _   ~   ~   ~   _   ~   ~   _   _   ~   ~  
 _   ~   _   ~   _   _   ~   _   _   _   _   ~   _   _   _   _   _   _   ~   _   _   _   _   ~  
 ~   _   ~   _   _   _   _   _   _   _   _   _   _   _   _   _   _   _   _   _   _   _   _   ~  
 _   _   _   _   _   _   _   _   ~   _   _   _   _   _   _   _   _   _   _   _   _   _   _   _  
 ~   ~   _   _   _   _   _   _   _   _   _   _   _   _   _   _   _   _   _   ~   ~   _   _   _  
 ~   _   _   _   _   _   _   _   _   _   _   _   _   _   _   ~   ~   _   _   _   ~   ~   _   ~  
 ~   _   _   _   _   _   _   _   _   _   _   _   _   _   _   ~   ~   _   _   _   _   _   _   _  
 ~   _   _   ~   _   _   _   _   _   _   _   _   _   _   _   _   _   _   _   _   _   _   _   ~  
 ~   _   _   ~   ~   _   _   _   _   _   _   _   _   _   ~   ~   ~   _   _   ~   _   _   ~   ~  
 ~   _   _   ~   _   _   _   _   _   _   _   _   _   _   ~   ~   _   _   ~   _   _   _   ~   ~  
 _   _   _   _   _   _   _   _

## Mountain Seeding

In [1836]:
for coord_x, row in enumerate(Map):
    for coord_y, terrain in enumerate(row):
        chance = random.uniform(0,1)
        if Map[coord_x][coord_y] == '_' and chance <= init_mountain_chance:
            Map[coord_x][coord_y] = 'M'

In [1837]:
map_print(Map)

 ~   ~   ~   ~   _   ~   ~   ~   ~   ~   ~   ~   ~   _   ~   ~   ~   _   ~   ~   _   _   ~   ~  
 _   ~   _   ~   _   _   ~   _   _   _   _   ~   _   _   _   _   _   _   ~   _   _   _   _   ~  
 ~   _   ~   _   _   _   _   _   _   _   _   _   _   _   M   _   _   _   _   _   _   _   _   ~  
 _   _   _   _   M   _   _   _   ~   _   _   M   _   _   _   _   _   _   _   _   _   _   _   _  
 ~   ~   _   _   _   _   _   _   _   _   _   _   M   _   _   _   _   _   _   ~   ~   _   _   _  
 ~   _   M   _   _   _   _   _   _   _   _   _   _   _   _   ~   ~   _   _   _   ~   ~   _   ~  
 ~   _   _   _   _   _   _   _   _   _   _   _   _   _   _   ~   ~   _   _   _   _   _   _   _  
 ~   _   _   ~   _   _   _   _   _   _   _   M   _   _   _   _   _   _   _   _   _   M   _   ~  
 ~   _   _   ~   ~   _   _   _   _   _   _   _   _   _   ~   ~   ~   _   _   ~   _   _   ~   ~  
 ~   _   _   ~   _   _   _   _   _   _   _   _   _   _   ~   ~   _   _   ~   _   _   _   ~   ~  
 _   _   _   _   _   _   _   _

## Mountain Expanding

In [1838]:
for i in range(0,mountain_amplifier):
    for coord_x, row in enumerate(Map):
        for coord_y, terrain in enumerate(row):     
            if Map[coord_x][coord_y] == '_':
                area = neighbors(Map, 1, coord_x, coord_y).copy()
                Map[coord_x][coord_y] = get_neighbour_mountain(area)  

In [1839]:
map_print(Map)

 ~   ~   ~   ~   _   ~   ~   ~   ~   ~   ~   ~   ~   _   ~   ~   ~   _   ~   ~   _   _   ~   ~  
 _   ~   _   ~   _   _   ~   _   _   _   _   ~   _   _   _   _   _   _   ~   _   _   _   _   ~  
 ~   _   ~   _   _   _   _   _   _   _   _   _   _   _   M   _   _   _   _   _   _   _   _   ~  
 _   _   _   _   M   _   _   _   ~   _   _   M   _   _   _   _   _   _   _   _   _   _   _   _  
 ~   ~   _   _   _   _   _   _   _   _   _   M   M   _   _   _   _   _   _   ~   ~   _   _   _  
 ~   _   M   _   _   _   _   _   _   _   _   M   _   _   _   ~   ~   _   _   _   ~   ~   _   ~  
 ~   _   _   _   _   _   _   _   _   _   _   M   _   _   _   ~   ~   _   _   _   _   _   _   _  
 ~   _   _   ~   _   _   _   _   _   _   _   M   M   _   _   _   _   _   _   _   _   M   _   ~  
 ~   _   _   ~   ~   _   _   _   _   _   _   _   _   _   ~   ~   ~   _   _   ~   _   _   ~   ~  
 ~   _   _   ~   _   _   _   _   _   _   _   _   _   _   ~   ~   _   _   ~   _   _   _   ~   ~  
 _   _   _   _   _   _   _   _

## Region Seeding

In [1840]:
count = 1
while(count <= number_of_region):
    for coord_x, row in enumerate(Map):
        for coord_y, terrain in enumerate(row):
            chance = random.uniform(0,1)
            if Map[coord_x][coord_y] == '_' and chance <= init_region_chance and count <= number_of_region:
                Map[coord_x][coord_y] = count
                count += 1

In [1841]:
map_print(Map)

 ~   ~   ~   ~   _   ~   ~   ~   ~   ~   ~   ~   ~   _   ~   ~   ~   _   ~   ~   _   _   ~   ~  
 _   ~   _   ~   _   _   ~   _   _   _   _   ~   _   _   _   _   _   _   ~   _   _   _   _   ~  
 ~   _   ~   _   _   _   _   _   _   _   _   _   _   _   M   _   _   _   _   _   _   _   _   ~  
 _   _   _   _   M   _   _   _   ~   _   _   M   _   _   _   _   _   _   _   _   _   _   _   _  
 ~   ~   _   _   _   _   _   _   _   _   _   M   M   _   _   _   _   _   _   ~   ~   _   _   _  
 ~   _   M   _   _   _   _   _   _   _   _   M   _   _   _   ~   ~   _   _   _   ~   ~   _   ~  
 ~   _   _   _   _   _   _   _   _   _   _   M   _   _   _   ~   ~   _   _   _   _   _   _   7  
 ~   _   _   ~   _   _   _   _   _   _   _   M   M   _   _   _   _   _   _   _   _   M   _   ~  
 ~   _   _   ~   ~   _   _   _   _   _   _   _   _   _   ~   ~   ~   _   _   ~   _   _   ~   ~  
 ~   _   _   ~   _   _   _   _   _   _   _   _   _   _   ~   ~   _   _   ~   _   _   _   ~   ~  
 _   _   _   _   _   _   _   _

## Region Expanding

In [1842]:
for i in range(0,(rows+cols) + 20):
    for coord_x, row in enumerate(Map):
        for coord_y, terrain in enumerate(row):     
            if Map[coord_x][coord_y] == '_':
                area = neighbors(Map, 1, coord_x, coord_y).copy()
                Map[coord_x][coord_y] = get_neighbour_region(area)

In [1843]:
map_print(Map)

 ~   ~   ~   ~   6   ~   ~   ~   ~   ~   ~   ~   ~   7   ~   ~   ~   7   ~   ~   7   7   ~   ~  
 6   ~   6   ~   6   6   ~   6   6   6   6   ~   7   7   7   7   7   7   ~   7   7   7   7   ~  
 ~   6   ~   6   6   6   6   6   6   6   6   6   7   7   M   7   7   7   7   7   7   7   7   ~  
 6   6   6   6   M   6   6   6   ~   6   6   M   7   7   7   7   7   7   7   7   7   7   7   7  
 ~   ~   6   6   6   6   6   6   6   6   6   M   M   7   7   7   7   7   7   ~   ~   7   7   7  
 ~   6   M   6   6   6   6   6   6   6   6   M   8   7   7   ~   ~   7   7   7   ~   ~   7   ~  
 ~   6   6   6   6   6   6   6   6   6   6   M   8   8   8   ~   ~   7   7   7   7   7   7   7  
 ~   6   6   ~   6   6   6   6   6   6   6   M   M   8   8   8   8   7   7   7   7   M   7   ~  
 ~   4   4   ~   ~   6   4   6   6   6   6   6   8   8   ~   ~   ~   8   7   ~   7   7   ~   ~  
 ~   4   4   ~   4   4   4   6   6   6   6   6   8   8   ~   ~   8   8   ~   7   7   7   ~   ~  
 4   4   4   4   4   4   4   6

## Prison Placing

In [1844]:
count = 1

while(count <= number_of_prison):
    for coord_x, row in enumerate(Map):
        for coord_y, terrain in enumerate(row):
            chance = random.uniform(0,1)
            if Map[coord_x][coord_y] in range(1,number_of_region + 1) and chance <= 0.001 and count <= number_of_prison:
                Map[coord_x][coord_y] = 'p'
                count += 1

pirate_placed = False
while(pirate_placed == False):
    for coord_x, row in enumerate(Map):
        for coord_y, terrain in enumerate(row):
            if Map[coord_x][coord_y] == 'p' and random.uniform(0,1) <= 1/number_of_prison and pirate_placed == False:
                Map[coord_x][coord_y] = 'P'
                pirate_placed = True

In [1845]:
agent_placed = False
while(agent_placed == False):
    for coord_x, row in enumerate(Map):
        for coord_y, terrain in enumerate(row):
            if Map[coord_x][coord_y] in range(1, number_of_region + 1) and random.uniform(0,1) <= 0.001 and agent_placed == False:
                Map[coord_x][coord_y] = 'A'
                agent_placed = True

In [1846]:
map_print(Map)

 ~   ~   ~   ~   6   ~   ~   ~   ~   ~   ~   ~   ~   7   ~   ~   ~   7   ~   ~   7   7   ~   ~  
 6   ~   6   ~   6   6   ~   6   6   6   6   ~   7   7   7   7   7   7   ~   7   7   7   7   ~  
 ~   6   ~   6   6   6   6   6   6   6   6   6   7   7   M   7   7   7   A   7   7   7   7   ~  
 6   6   6   6   M   6   6   6   ~   6   6   M   7   7   7   7   7   7   7   7   7   7   7   7  
 ~   ~   6   6   6   6   6   6   6   6   6   M   M   7   7   7   7   7   7   ~   ~   7   7   7  
 ~   6   M   6   6   6   6   6   6   6   6   M   8   7   7   ~   ~   7   7   7   ~   ~   7   ~  
 ~   6   6   6   6   6   6   6   6   6   6   M   8   8   8   ~   ~   7   7   7   7   7   7   7  
 ~   6   6   ~   6   6   6   6   6   6   6   M   M   8   8   8   8   7   7   7   7   M   p   ~  
 ~   4   4   ~   ~   6   4   6   6   6   6   6   8   8   ~   ~   ~   8   7   ~   7   7   ~   ~  
 ~   4   4   ~   4   4   4   6   6   6   6   6   8   8   ~   ~   8   8   ~   7   7   7   ~   ~  
 4   4   4   4   4   4   4   6